# ISMB figures notebook

In [6]:

include("engines/init.jl")
include("engines/data_processing.jl")
include("engines/deep_learning.jl")
include("engines/cross_validation.jl")
outpath, session_id = set_dirs() ;

  Activating project at `~/vae_cox`


In [ ]:
BRCA = MLSurvDataset("Data/TCGA_BRCA_tpm_n1049_btypes_labels_surv.h5")

In [ ]:
LAML = MLSurvDataset("Data/LGN_AML_tpm_n300_btypes_labels_surv.h5") 

In [ ]:
BRCA_pcoding = BRCA.biotypes .== "protein_coding"
println(sum(BRCA_pcoding))
println(size(BRCA.samples)[1])
println(mean(BRCA.surve .!= 0))


In [ ]:
pcoding = [occursin("protein_coding", bt) for bt in LAML.biotypes]
println(size(LAML.genes[pcoding])[1])
println(mean(LAML.surve .!= 0))

In [ ]:
# testing VAE-Cox
# set hyper parameters 
DATA = BRCA
keep = BRCA_pcoding
# split train test
folds = split_train_test(Matrix(DATA.data[:,keep]), DATA.survt, DATA.surve, DATA.samples;nfolds =5)


In [ ]:
device!()

In [3]:
# testing VAE-Cox
# set hyper parameters 
DATA,nfolds, nepochs, dim_redux = LAML, 5, 100, 125
keep = [occursin("protein_coding", bt) for bt in DATA.biotypes]
println(sum(keep))
println(size(DATA.samples)[1])
println(mean(DATA.surve .!= 0))
params_dict = Dict(
        ## run infos 
        "session_id" => session_id, "nfolds" =>5,  "modelid" => "$(bytes2hex(sha256("$(now())"))[1:Int(floor(end/3))])",
        "machine_id"=>strip(read(`hostname`, String)), "device" => "$(device())", "model_title"=>"AECPHDNN",
        ## data infos 
        "dataset" => "BRCA_data(norm=true)", "nsamples" => size(DATA.samples)[1],
        "nsamples_test" => Int(round(size(DATA.samples)[1] / nfolds)), "ngenes" => size(DATA.genes[keep])[1],
        "nsamples_train" => size(DATA.samples)[1] - Int(round(size(DATA.samples)[1] / nfolds)),
        ## optim infos 
        "nepochs" => nepochs, "ae_lr" =>1e-5, "cph_lr" => 1e-5, "ae_wd" => 1e-6, "cph_wd" => 1e-6,
        ## model infos
        "model_type"=> "aecphdnn", "dim_redux" => dim_redux, "ae_nb_hls" => 2, "ae_hl_size"=> 128,
        "enc_nb_hl" => 2, "enc_hl_size"=> 128,  "dec_nb_hl" => 2 , "dec_hl_size"=> 128,
        "nb_clinf" => 0, "cph_nb_hl" => 2, "cph_hl_size" => 64, 
        "insize" => size(DATA.genes[keep])[1],
        ## metrics
        "model_cv_complete" => false
    )
# split train test
folds = split_train_test(Matrix(DATA.data[:,keep]), DATA.survt, DATA.surve, DATA.samples;nfolds =5)
fold = folds[1]
# format input data  
train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst = format_train_test(fold)
# create model 
#model = build_vaecox(params_dict)
# train model 
## gradient CPH            

# report learning curves
# test model
# report c-index

0.7433333333333333


(Float32[1.5314789 1.4358444 … 1.513617 1.6565772; 0.5786392 0.94497585 … 0.6857417 0.8350561; … ; 0.0 0.0 … 0.0 0.0; 0.06069783 0.049218025 … 0.0 0.0], [6033 5912 … 11 8], [0 0 … 1 1], 0.005555555555555556, Float32[1.5194342 1.7227983 … 1.66483 1.7653705; 0.81224465 0.8813847 … 1.1367205 0.53655845; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.096910015 … 0.053078443 0.07188203], [5918 5848 … 14 9], [0 0 … 1 1], 0.023255813953488372)

In [14]:
function VAE_COX_loss(VENC, CPH, X, Y_e, NE_frac;device = gpu)
    mu, log_sigma = VENC(X)
    z = mu + device(randn(Float32, size(log_sigma))) .* exp.(log_sigma)
    outs = vec(CPH(z))
    hazard_ratios = exp.(outs)
    log_risk = log.(cumsum(hazard_ratios))
    uncensored_likelihood = outs .- log_risk
    censored_likelihood = uncensored_likelihood .* Y_e'
    #neg_likelihood = - sum(censored_likelihood) / sum(e .== 1)
    neg_likelihood = - sum(censored_likelihood) * NE_frac
    return neg_likelihood
end 

VAE_COX_loss (generic function with 1 method)

In [31]:
function validate_vaecox(DATA, params_dict;device = gpu)
    nfolds, nepochs, dim_redux = 5, 10000, 125
    keep = [occursin("protein_coding", bt) for bt in DATA.biotypes]
    # split train test
    folds = split_train_test(Matrix(DATA.data[:,keep]), DATA.survt, DATA.surve, DATA.samples;nfolds =5)
    fold = folds[1]

    venc = VariationalEncoder(size(DATA.data[:,keep])[2], 125, 600)
    vdec = Decoder(size(DATA.data[:,keep])[2], 125, 600)
    VAE_opt = Flux.ADAM(1e-6)

    cphdnn = device(Chain(Dense(125, 100,leakyrelu),Dense(100, 1)))
    cphdnn_opt = Flux.ADAM(1e-5)

    train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst = format_train_test(fold)
    for i in 1:nepochs
        ps1 = Flux.params(venc, vdec)
        gs1 = gradient(ps1) do
            VAE_lossf(venc, vdec, train_x)
        end 
        VAE_loss = VAE_lossf(venc, vdec, train_x)
        VAE_cor = round(my_cor(vec(train_x), vec(MyReconstruct(venc, vdec, train_x)[end])),digits = 3)
        VAE_test = round(my_cor(vec(test_x), vec(MyReconstruct(venc, vdec, test_x)[end])),digits = 3)

        ps2 = Flux.params(venc, cphdnn)
        gs2 = gradient(ps2) do 
            VAE_COX_loss(venc, cphdnn, train_x, train_y_e, NE_frac_tr)
        end 
        CPH_loss = round(VAE_COX_loss(venc, cphdnn, train_x, train_y_e, NE_frac_tr), digits = 3)
        mu, log_sigma = venc(train_x)
        z = mu + device(randn(Float32, size(log_sigma))) .* exp.(log_sigma)
        OUTS_tr = vec(cphdnn(z))
        cind_tr, cdnt_tr, ddnt_tr, tied_tr  = concordance_index(train_y_t, train_y_e, OUTS_tr)

        Flux.update!(VAE_opt, ps1, gs1)
        Flux.update!(cphdnn_opt, ps2, gs2)
        if i % 100 == 0 || i == 1
            println("$i TRAIN - VAE-loss-avg: $VAE_loss\tVAE-cor: $VAE_cor CPH-loss: $CPH_loss CPH-cind: $(round(cind_tr,digits=3))\tTEST - VAE-cor: $VAE_test")
        end
    end
end 

validate_vaecox (generic function with 1 method)

In [32]:
validate_vaecox(DATA, params_dict)

1 TRAIN - VAE-loss-avg: 60284.832	VAE-cor: 0.006 CPH-loss: 35.594 CPH-cind: 0.504	TEST - VAE-cor: 0.006


100 TRAIN - VAE-loss-avg: 61348.703	VAE-cor: 0.033 CPH-loss: 45.128 CPH-cind: 0.469	TEST - VAE-cor: 0.036


200 TRAIN - VAE-loss-avg: 52931.45	VAE-cor: 0.062 CPH-loss: 34.331 CPH-cind: 0.496	TEST - VAE-cor: 0.062


300 TRAIN - VAE-loss-avg: 48749.258	VAE-cor: 0.096 CPH-loss: 34.436 CPH-cind: 0.467	TEST - VAE-cor: 0.096


400 TRAIN - VAE-loss-avg: 38963.156	VAE-cor: 0.127 CPH-loss: 23.187 CPH-cind: 0.514	TEST - VAE-cor: 0.132


...

8500 TRAIN - VAE-loss-avg: 13649.807	VAE-cor: 0.874 CPH-loss: 5.538 CPH-cind: 0.529	TEST - VAE-cor: 0.873


8600 TRAIN - VAE-loss-avg: 13686.388	VAE-cor: 0.878 CPH-loss: 5.575 CPH-cind: 0.511	TEST - VAE-cor: 0.853


8700 TRAIN - VAE-loss-avg: 13662.048	VAE-cor: 0.879 CPH-loss: 5.34 CPH-cind: 0.508	TEST - VAE-cor: 0.867


8800 TRAIN - VAE-loss-avg: 13626.84	VAE-cor: 0.884 CPH-loss: 5.526 CPH-cind: 0.518	TEST - VAE-cor: 0.864


8900 TRAIN - VAE-loss-avg: 13553.521	VAE-cor: 0.881 CPH-loss: 5.425 CPH-cind: 0.495	TEST - VAE-cor: 0.872


In [ ]:
for iter in 1:nepochs
    ps1 = Flux.params(model["cph"].model, model["enc"])
    gs1 = gradient(ps1) do
        model["cph"].lossf(model["cph"],model["enc"], train_x, train_y_e, NE_frac_tr, params_dict["cph_wd"])
    end 
    ## gradient Auto-Encoder 
    ps2 = Flux.params(model["ae"].net)
    gs2 = gradient(ps2) do
        model["ae"].lossf(model["ae"], train_x, train_x, weight_decay = params_dict["ae_wd"])
    end
    Flux.update!(model["cph"].opt, ps1, gs1)
    Flux.update!(model["ae"].opt, ps2, gs2)

    ######
    OUTS_tr = vec(model["cph"].model(model["enc"](train_x)))
    ae_loss = model["ae"].lossf(model["ae"], train_x, train_x, weight_decay = params_dict["ae_wd"])
    ae_cor =  round(my_cor(vec(train_x), vec(model["ae"].net(train_x))),digits = 3)
    cph_loss = model["cph"].lossf(model["cph"],model["enc"](train_x), train_y_e, NE_frac_tr, params_dict["cph_wd"])
    ae_loss_test = round(model["ae"].lossf(model["ae"], test_x, test_x, weight_decay = params_dict["ae_wd"]), digits = 3)
    ae_cor_test = round(my_cor(vec(test_x), vec(model["ae"].net(test_x))), digits= 3)
    cph_loss_test = round(model["cph"].lossf(model["cph"],model["enc"](test_x), test_y_e, NE_frac_tst, params_dict["cph_wd"]), digits= 3)
                    
    OUTS_tst =  vec(model["cph"].model(model["enc"](test_x)))
            
    cind_tr, cdnt_tr, ddnt_tr, tied_tr  = concordance_index(train_y_t, train_y_e, OUTS_tr)
    cind_test,cdnt_tst, ddnt_tst, tied_tst = concordance_index(test_y_t, test_y_e,OUTS_tst)
    if iter % 10 == 0       
        println("FOLD $(fold["foldn"]) $iter\t TRAIN AE-loss $(round(ae_loss,digits =3)) \t AE-cor: $(round(ae_cor, digits = 3))\t cph-loss-avg: $(round(cph_loss / params_dict["nsamples_train"],digits =6)) \t cph-cind: $(round(cind_tr,digits =3))")
        println("\t\tTEST AE-loss $(round(ae_loss_test,digits =3)) \t AE-cor: $(round(ae_cor_test, digits = 3))\t cph-loss-avg: $(round(cph_loss_test / params_dict["nsamples_test"],digits =6)) \t cph-cind: $(round(cind_test,digits =3)) [$(Int(cdnt_tst)), $(Int(ddnt_tst)), $(Int(tied_tst))]")
    end
end 